In [77]:
# 讀檔
import csv
with open('movies.csv', encoding = 'utf8') as csvfile:
    # 解析內容
    rows = list(csv.reader(csvfile))

In [78]:
test_movie = []
train_movie = []
for movie in rows[1:]:
    if movie[2] != 'NA':
        if len(test_movie) < 500:
            test_movie.append(movie)
        else:
            train_movie.append(movie)

print(test_movie[0])
print(train_movie[0])

#只取第一個分類
for movie in test_movie:
    temp = movie[2].split(",")
    movie[2] = temp[0]

for movie in train_movie:
    temp = movie[2].split(",")
    movie[2] = temp[0]

print(test_movie[0])
print(train_movie[0])

['0', '一世狂野', '劇情,犯罪,歷史/傳記', '喬治戎格一生都在追求所謂的美國夢，也就是享受美好富裕的生活，但是他卻不願像他父親那樣一輩子都只是個出賣勞力的建築工人。於是他搬到陽光明媚的加州，靠著販賣大麻賺錢，起初，他販毒只是為了享受自由自在的生活，但是當他野心越來越大，他的勢力也日益坐大之際，卻在此時被捕入獄。 他在牢裡認識一個能言善道，自稱熟識哥倫比亞販毒集團的牢友狄亞哥，他出獄後果真把當時勢力最大的毒梟艾斯科巴介紹給喬治認識，艾斯科巴計畫將古柯鹼大量引進美國的迪斯可舞廳，希望能引領一股吸毒狂歡的風潮。除了毒品供應商之外，狄亞哥也介紹了一個美艷又狂野的女人瑪莎給喬治，他們瘋狂相愛，之後馬莎還替他生下一個可愛的女兒克莉絲汀娜，也是喬治一生的最愛。喬治很快就靠著販毒發大財，他還得買一棟大房子專門存放每天賺進來的大把鈔票，但是日進斗金卻整天提心吊膽的生活卻讓喬治開始省思，到底他要繼續過著揮霍富裕的生活，還是為了自己心愛的女兒應該轉性投資正當的事業？可是這時聯邦調查局的探員，也開始盯上毒源禍首的喬治……', '2001-10-12']
['500', '浪漫醫生', '劇情,喜劇,愛情', '李察基爾在這部充滿溫馨的喜劇電影中飾演一個百分百的男人，一輩子生活在溫柔鄉中，享受著甜蜜的愛情，令世間的男人羨慕不已。原來李察基爾是美國一個富有而成功的婦科醫生，圍繞在他工作及家庭周圍的都是女人，他成天沐浴在女人世界裡，他的宗教信仰是女人，他崇拜的也是女人，他把一切都貢獻給了女人。可是接踵而來的卻是一大堆有關女人的煩惱；先是診所裡排得過於擁擠的醫療行程令他疲累不堪；接著是太太為了張羅女兒的婚事讓全家變得緊張兮兮；最後又是小姨子帶了三個女兒借宿在他家，這使得一向平靜的家中憑添了許多煩惱事。李察基爾為了刻意躲避這些女人經，決心遠離塵世，驅車前往鄉間高爾夫俱樂部避難，不料，他的生命中又加添了一位樂觀，風趣而隨和的球友，不過這個女人帶給他的卻是快樂無比的情愛生活。', '2001-05-26']
['0', '一世狂野', '劇情', '喬治戎格一生都在追求所謂的美國夢，也就是享受美好富裕的生活，但是他卻不願像他父親那樣一輩子都只是個出賣勞力的建築工人。於是他搬到陽光明媚的加州，靠著販賣大麻賺錢，起初，他販毒只是為了享受自由自在的生活，但是當他野心越來越大，他的勢力也日益坐大之

In [79]:
#有哪些分類
categories = []

for movie in train_movie:
    temp = movie[2].split(",")
    categories.append(temp[0])

categories = set(categories)
categories = list(categories)
print(categories)

['勵志', '武俠', '音樂/歌舞', '戰爭', '歷史/傳記', '動作', '紀錄片', '溫馨/家庭', '愛情', '奇幻', '科幻', '動畫', '懸疑/驚悚', '戲劇', '恐怖', '冒險', '犯罪', '劇情', '影展', '喜劇']


In [80]:
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import math
from sklearn.preprocessing import normalize

In [97]:
vectorizer = TfidfVectorizer(sublinear_tf=False, stop_words=None, token_pattern="(?u)\\b\\w+\\b", smooth_idf=True, norm='l2')
train_data = []
for i in train_movie:
    train_data.append(i[3][20:23])
tfidf = vectorizer.fit_transform(train_data)
tfidf = tfidf.toarray()
#df_tfidf = pd.DataFrame(tfidf.toarray(),columns=vectorizer.get_feature_names(), index=range(len(train_data))).transpose()

#df_tfidf
#print("TFIDF")

MemoryError: Unable to allocate 350. MiB for an array with shape (7354, 6236) and data type float64